<br>
This is exploring and modifying PyTorch/Pyro's bayesian regression capabilities; heavily borrowing from their provided example notebook, available at: http://pyro.ai/examples/bayesian_regression.html

I'm going to scale up the toy data set size, for learning purposes. Namely, the number of features (1 -> 10) and the number of data points (100 -> 1000)

<br>
# 1. Regular regression part
**Note: I've scaled this up to multiple variables (10), the original tutorial only had 1 X-variable**

## Importing the libraries

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
# for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

<br>
## Defining functions and parameters for creating an experimenting data set

In [2]:
N = 1000  # size of toy data

def build_linear_dataset(N, p = 10, noise_std=0.01):
    X = np.random.rand(N, p)
    # w = 3
    w = 3 * np.ones(p)
    # b = 1
    y = np.matmul(X, w) + np.repeat(1, N) + np.random.normal(0, noise_std, size=N)
    y = y.reshape(N, 1)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    assert data.shape == (N, p + 1)
    return data

<br>
## Defining the simple regression model

In [3]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)

    def forward(self, x):
        return self.linear(x)

regression_model = RegressionModel(10)

<br>
## Defining the training parameters & function, then training
Also printing loss and learned parameters

I bumped:
1. the number of training epochs from 1000 -> 10000
2. the printing freq from 50 -> 250

In [4]:
# Defining loss
loss_fn = torch.nn.MSELoss(size_average = False)
# Defining optimizer
optim = torch.optim.Adam(regression_model.parameters(), lr = 0.05)
# Defining training length
num_iterations = 10000 if not smoke_test else 2

# Defining function to train
def main():
    # Generating the data
    data = build_linear_dataset(N)
    # Defining X
    x_data = data[:, :-1]
    # Defining Y
    y_data = data[:, -1]
    # Training loop
    for j in range(num_iterations):
        # run the model forward on the data
        y_pred = regression_model(x_data).squeeze(-1)
        # calculate the mse loss
        loss = loss_fn(y_pred, y_data)
        # initialize gradients to zero
        optim.zero_grad()
        # backpropagate
        loss.backward()
        # take a gradient step
        optim.step()
        # Print loss every 50 iterations
        if (j + 1) % 250 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))
    # Inspect learned parameters
    print("\nLearned parameters:")
    for name, param in regression_model.named_parameters():
        print("%s: " % (name) + str(param.data.numpy()))

if __name__ == '__main__':
    main()

[iteration 0250] loss: 114.1863
[iteration 0500] loss: 86.1122
[iteration 0750] loss: 62.4182
[iteration 1000] loss: 42.9902
[iteration 1250] loss: 27.5911
[iteration 1500] loss: 16.3051
[iteration 1750] loss: 8.8034
[iteration 2000] loss: 4.3176
[iteration 2250] loss: 1.9235
[iteration 2500] loss: 0.7948
[iteration 2750] loss: 0.3308
[iteration 3000] loss: 0.1670
[iteration 3250] loss: 0.1183
[iteration 3500] loss: 0.1063
[iteration 3750] loss: 0.1039
[iteration 4000] loss: 0.1036
[iteration 4250] loss: 0.1035
[iteration 4500] loss: 0.1035
[iteration 4750] loss: 0.1035
[iteration 5000] loss: 0.1035
[iteration 5250] loss: 0.1035
[iteration 5500] loss: 0.1035
[iteration 5750] loss: 0.1035
[iteration 6000] loss: 0.1035
[iteration 6250] loss: 0.1035
[iteration 6500] loss: 0.1035
[iteration 6750] loss: 0.1035
[iteration 7000] loss: 0.1035
[iteration 7250] loss: 0.1035
[iteration 7500] loss: 0.1035
[iteration 7750] loss: 0.1035
[iteration 8000] loss: 0.1035
[iteration 8250] loss: 0.1035
[it

The true (generated) parameters were 3 and 1, with a noise of $\sigma = 0.01$

**Possible typos in Pyro documentation**:

1. They say that $\sigma = 0.1$
2. They say they used a learning rate of $0.01$
2. They say they train for $500$ iterations

None of the above seem to agree with their code

# 2. Bayesian part

Minimally modified from Pyro's documentation:

`random_module()` effectively takes a given `nn.Module` (from PyTorch) and turns it into a distribution over the same module; in our case, this will be a distribution over regressors. Specifically, each parameter in the original regression model is sampled from the provided prior. This allows us to repurpose vanilla regression models for use in the Bayesian setting.

## Toy example - simpler than the next header's

In [5]:
# Defining a standard normal distribution prior
# We need to feed the distribution tensors, rather than ints/floats
loc = torch.zeros(1, 1)
scale = torch.ones(1, 1)
# Recall that "Normal" is from pyro.distributions
prior = Normal(loc, scale)

# overload the parameters in the regression module with samples from the prior
Bayesian_Model = pyro.random_module("abitrary_name",
                                     regression_model,     # The PyTorch model
                                     prior)                # The Pyro prior
# sample a regressor from the prior
sampled_reg_model = Bayesian_Model()

<br>
## Defining the model, which we will train
Another possible code comment typo: their comments say unit normal

In [6]:
def model(data):
    
    
    # Create wide normal priors - mu = 0, sigma = 10
    weights_loc, weights_scale = torch.zeros(1, 10), (10 * torch.ones(1, 10))
    # For the bias, defining mu and sigma
    bias_loc, bias_scale = torch.zeros(1), 10 * torch.ones(1)
    
    
    # For the weight, creating and assigning the prior distribution
    w_priors = Normal(weights_loc, weights_scale).independent(1)
    # For the bias, creating and assigning the prior distribution
    b_prior = Normal(bias_loc, bias_scale).independent(1)
    
    
    # Putting those into a dictionary
    priors = {'linear.weights': w_priors,
              'linear.bias': b_prior}
    
    
    # lift module parameters to random variables sampled from the priors
    # This is like creating the nn.Module (analogy to PyTorch)
    lifted_module = pyro.random_module("module", regression_model, priors)
    # sample a regressor (which also samples w and b)
    # This is like calling the nn.Module (analogy to PyTorch)
    lifted_reg_model = lifted_module()
    
    
    with pyro.iarange("map", N):
        x_data = data[:, :-1]
        y_data = data[:, -1]

        # run the regressor forward conditioned on data
        prediction_mean = lifted_reg_model(x_data).squeeze(-1)
        
        # condition on the observed data
        pyro.sample("obs",
                    Normal(prediction_mean, 0.1 * torch.ones(data.size(0))),
                    obs=y_data)

The guide serves the purpose of having all the trainable distributions

In [7]:
softplus = torch.nn.Softplus()

def guide(data):
    
    
    # define our variational parameters
    w_locs = torch.randn(1, 10)
    # note that we initialize our scales to be pretty narrow
    w_log_sigs = torch.tensor(-3.0 * torch.ones(1, 10) + 0.05 * torch.randn(1, 10))
    b_loc = torch.randn(1, 1)
    b_log_sig = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    
    
    # register learnable params in the param store
    mw_params = pyro.param("guide_mean_weights", w_locs)
    sw_params = softplus(pyro.param("guide_log_scale_weights", w_log_sigs))
    mb_param = pyro.param("guide_mean_bias", b_loc)
    sb_param = softplus(pyro.param("guide_log_scale_bias", b_log_sig))
    
    # guide distributions for w and b
    w_dists = Normal(mw_params, sw_params).independent(1)
    b_dist = Normal(mb_param, sb_param).independent(1)
    dists = {'linear.weights': w_dists,
             'linear.bias': b_dist}
    
    
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    # sample a regressor (which also samples w and b)
    return lifted_module()

In [8]:
optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [9]:
def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N)
    for j in range(num_iterations):
        # calculate the loss and take a gradient step
        loss = svi.step(data)
        if (j) % 250 == 0:
            print("[iteration %04f] loss: %.4f" % ((j+1), loss / float(N)))

if __name__ == '__main__':
    main()

[iteration 1.000000] loss: 8.5390
[iteration 251.000000] loss: -1.3000
[iteration 501.000000] loss: -1.3423
[iteration 751.000000] loss: -1.3663
[iteration 1001.000000] loss: -1.3719
[iteration 1251.000000] loss: -1.3253
[iteration 1501.000000] loss: -1.3678
[iteration 1751.000000] loss: -1.3617
[iteration 2001.000000] loss: -1.3683
[iteration 2251.000000] loss: -1.2813
[iteration 2501.000000] loss: -1.3698
[iteration 2751.000000] loss: -1.3619
[iteration 3001.000000] loss: -1.3578
[iteration 3251.000000] loss: -1.3544
[iteration 3501.000000] loss: -1.3675
[iteration 3751.000000] loss: -1.3672
[iteration 4001.000000] loss: -1.3683
[iteration 4251.000000] loss: -1.3204
[iteration 4501.000000] loss: -1.3286
[iteration 4751.000000] loss: -1.3659
[iteration 5001.000000] loss: -1.3699
[iteration 5251.000000] loss: -1.3693
[iteration 5501.000000] loss: -1.3697
[iteration 5751.000000] loss: -1.3693
[iteration 6001.000000] loss: -1.3174
[iteration 6251.000000] loss: -1.3700
[iteration 6501.000

In [10]:
for name in pyro.get_param_store().get_all_param_names():
    print("\n[%s]: " % (name) + str(pyro.param(name).data.numpy()))


[guide_mean_weights]: [[ 0.9496766   0.03266967  0.6193119  -3.3383305  -0.53291076 -0.4834933
   0.0514564  -1.1796219  -0.01656455 -0.18051535]]

[guide_log_scale_weights]: [[-3.0165079 -2.9443128 -3.0165148 -2.9527166 -2.9659028 -3.0106964
  -3.0520527 -3.0153282 -2.9680774 -3.018161 ]]

[guide_mean_bias]: [[1.0038915]]

[guide_log_scale_bias]: [[-5.7794676]]

[module$$$linear.weight]: [[3.0055509 3.0038602 3.0039666 3.0029285 3.0065365 3.0043755 3.003794
  3.0049517 3.0039704 3.0012436]]


In [11]:
one_x = np.linspace(6, 7, num = 20)
Xs = np.repeat(one_x[:, np.newaxis], 10, axis = 1)


y =  np.sum(3*Xs, axis = 1) + 1
y = y.reshape((20, 1))


x_data, y_data = torch.tensor(Xs).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)


loss = nn.MSELoss()
   
    
y_preds = torch.zeros(20, 1)


num_samples_of_param = 100
for i in range(num_samples_of_param):
    # guide does not require the data
    sampled_reg_model = guide(None)
    
    # run the regression model and add prediction to total
    y_preds = y_preds + sampled_reg_model(x_data)
# take the average of the predictions
y_preds = y_preds / num_samples_of_param
print ("Loss: ", loss(y_preds, y_data).item())

Loss:  0.07375814020633698


<br>
# Part 3. Bayesian with Hidden Layers

In [12]:
class FeedForwardNN(nn.Module):
    def __init__(self, p):
        # p = number of features
        super(FeedForwardNN, self).__init__()
        self.linear1 = nn.Linear(p, p)
        self.linear2 = nn.Linear(p, 1)

    def forward(self, x):
        first_layer_out = self.linear1(x)
        second_layer_out = self.linear2(x)
        return(second_layer_out)

feedforward_nn = FeedForwardNN(10)

In [13]:
def model(data):
    
    
    # First layer
    #    We have 10 input variables that go to 10 hidden units (10*10)
    weights_loc_first, weights_scale_first = torch.zeros(1, 10*10), (10 * torch.ones(1, 10*10))
    #    We have 1 intercept per hidden unit (1*10)
    bias_loc_first, bias_scale_first = torch.zeros(1, 1*10), 10 * torch.ones(1, 1*10)
    
    # Second layer
    #    We have 10 hidden activations that go to 1 output unit
    weights_loc_second, weights_scale_second = torch.zeros(1, 1*10), (10 * torch.ones(1, 1*10))
    #    We have 1 intercept per hidden unit (1*1)
    bias_loc_second, bias_scale_second = torch.zeros(1, 1*1), 10 * torch.ones(1, 1*1)
    
    
    # For the first layer
    #    Creating and assigning the prior distributions for the weights
    w_priors_first = Normal(weights_loc_first, weights_scale_first).independent(1)
    #    Creating and assigning the prior distribution for the intercepts
    b_prior_first = Normal(bias_loc_first, bias_scale_first).independent(1)
    
    # For the second layer
    #    Creating and assiging the prior distributions for the weights
    w_priors_second = Normal(weights_loc_second, weights_scale_second).independent(1)
    #    Creating and assigning the prior distribution for the intercepts
    b_prior_second = Normal(bias_loc_second, bias_scale_second).independent(1)
    
    
    # Putting those into a dictionary
    priors = {'linear.weights_first': w_priors_first,
              'linear.bias_first': b_prior_first,
              'linear.weights_second': w_priors_second,
              'linear.bias_second': b_prior_second}
    
    
    # lift module parameters to random variables sampled from the priors
    # This is like creating the nn.Module (analogy to PyTorch)
    lifted_module = pyro.random_module("module", feedforward_nn, priors)
    # sample a regressor (which also samples w and b)
    # This is like calling the nn.Module (analogy to PyTorch)
    lifted_nn_model = lifted_module()
    
    
    with pyro.iarange("map", N):
        x_data = data[:, :-1]
        y_data = data[:, -1]

        # run the regressor forward conditioned on data
        prediction_mean = lifted_nn_model(x_data).squeeze(-1)
        
        # condition on the observed data
        pyro.sample("obs",
                    Normal(prediction_mean, 0.1 * torch.ones(data.size(0))),
                    obs=y_data)

In [14]:
softplus = torch.nn.Softplus()

def guide(data):
    
    
    # First layer
    #     The weights
    w_locs_first = torch.randn(1, 10*10)
    w_log_sigs_first = torch.tensor(-3.0 * torch.ones(1, 10*10) + 0.05 * torch.randn(1, 10*10))
    #     The intercepts 
    b_loc_first = torch.randn(1, 1*10)
    b_log_sig_first = torch.tensor(-3.0 * torch.ones(1, 1*10) + 0.05 * torch.randn(1, 1*10))
    
    # Second layer
    #     The weights
    w_locs_second = torch.randn(1, 10*10)
    w_log_sigs_second = torch.tensor(-3.0 * torch.ones(1, 10*10) + 0.05 * torch.randn(1, 10*10))
    #     The intercepts 
    b_loc_second = torch.randn(1, 1*10)
    b_log_sig_second = torch.tensor(-3.0 * torch.ones(1, 1*10) + 0.05 * torch.randn(1, 1*10))
    
    
    # Register learnable params in the param store
    # First layer
    #    The weights
    mw_params_first = pyro.param("guide_mean_weights_first", w_locs_first)
    sw_params_first = softplus(pyro.param("guide_log_scale_weights_first", w_log_sigs_first))
    #    The intercepts
    mb_param_first = pyro.param("guide_mean_bias_first", b_loc_first)
    sb_param_first = softplus(pyro.param("guide_log_scale_bias_first", b_log_sig_first))
    
    # Second layer
    #    The weights
    mw_params_second = pyro.param("guide_mean_weights_second", w_locs_second)
    sw_params_second = softplus(pyro.param("guide_log_scale_weights_second", w_log_sigs_second))
    #    The intercepts
    mb_param_second = pyro.param("guide_mean_bias_second", b_loc_second)
    sb_param_second = softplus(pyro.param("guide_log_scale_bias_second", b_log_sig_second))
    
    
    # guide distributions for w and b
    # First layer
    w_dists_first = Normal(mw_params_first, sw_params_first).independent(1)
    b_dist_first = Normal(mb_param_first, sb_param_first).independent(1)
    # Second layer
    w_dists_second = Normal(mw_params_second, sw_params_second).independent(1)
    b_dist_second = Normal(mb_param_second, sb_param_second).independent(1)
    dists = {'linear.weights_first': w_dists_first,
             'linear.bias_first': b_dist_first,
             'linear.weights_second': w_dists_second,
             'linear.bias_second': b_dist_second}
    
    
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", feedforward_nn, dists)
    # sample a regressor (which also samples w and b)
    return lifted_module()

In [15]:
optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [16]:
def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N)
    for j in range(num_iterations):
        # calculate the loss and take a gradient step
        loss = svi.step(data)
        if (j + 1) % 250 == 0:
            print("[iteration %04f] loss: %.4f" % ((j+1), loss / float(N)))

if __name__ == '__main__':
    main()

[iteration 250.000000] loss: 2.9096
[iteration 500.000000] loss: 1.9006
[iteration 750.000000] loss: 1.0141
[iteration 1000.000000] loss: 0.2720
[iteration 1250.000000] loss: -0.3197
[iteration 1500.000000] loss: -0.7541
[iteration 1750.000000] loss: -1.0430
[iteration 2000.000000] loss: -1.2159
[iteration 2250.000000] loss: -1.3082
[iteration 2500.000000] loss: -1.3518
[iteration 2750.000000] loss: -1.3697
[iteration 3000.000000] loss: -1.3760
[iteration 3250.000000] loss: -1.3779
[iteration 3500.000000] loss: -1.3784
[iteration 3750.000000] loss: -1.3785
[iteration 4000.000000] loss: -1.3785
[iteration 4250.000000] loss: -1.3785
[iteration 4500.000000] loss: -1.3785
[iteration 4750.000000] loss: -1.3785
[iteration 5000.000000] loss: -1.3785
[iteration 5250.000000] loss: -1.3785
[iteration 5500.000000] loss: -1.3785
[iteration 5750.000000] loss: -1.3785
[iteration 6000.000000] loss: -1.3785
[iteration 6250.000000] loss: -1.3785
[iteration 6500.000000] loss: -1.3785
[iteration 6750.000

In [17]:
for name in pyro.get_param_store().get_all_param_names():
    print("\n[%s]: " % (name) + str(pyro.param(name).data.numpy()))


[guide_mean_weights_first]: [[ 0.1350662  -0.71258014  0.56413245 -1.0877305  -0.71924245 -0.18485934
   0.08444668 -0.62668455 -1.5295157   0.14656499 -0.80894274  0.16689745
  -0.38625666 -0.648746    0.7074361  -0.30794692 -0.7600678   0.4224629
   0.18932785 -2.457124   -0.01532408 -0.76843655 -0.15489753 -2.3491802
   1.2575742  -1.3192716   0.94393873  0.6760205   0.5581235   0.88240635
   0.4687971  -1.9414772  -0.7755105  -0.6907118   1.1868346   0.3619952
  -1.7380005   0.8275028  -1.0032492  -0.01862479 -0.14826553 -0.3772825
   1.8741345  -0.78002036  0.97561646 -0.8535137  -0.5274611  -0.36480266
   0.5219727   1.2725877  -0.30370033  0.992638    0.22331089 -1.213343
   0.4647048  -1.3845404  -0.66757077 -0.9428688   0.66205996  0.17044587
   0.52112764 -0.05161944 -0.34483764  1.6823118  -1.551885    0.21137127
   0.07317776  0.78594923  0.05508348  1.5101274  -0.6225961  -0.3489961
   0.30410987  0.6962635  -1.2059362  -1.5425351  -1.6297977  -0.8904026
  -0.5457716  -0.

In [18]:
one_x = np.linspace(6, 7, num = 20)
Xs = np.repeat(one_x[:, np.newaxis], 10, axis = 1)


y =  np.sum(3*Xs, axis = 1) + 1
y = y.reshape((20, 1))


x_data, y_data = torch.tensor(Xs).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)


loss = nn.MSELoss()
   
    
y_preds = torch.zeros(20, 1)


num_samples_of_param = 100
for i in range(num_samples_of_param):
    # guide does not require the data
    sampled_reg_model = guide(None)
    
    # run the regression model and add prediction to total
    y_preds = y_preds + sampled_reg_model(x_data)
# take the average of the predictions
y_preds = y_preds / num_samples_of_param
print ("Loss: ", loss(y_preds, y_data).item())

Loss:  2.251821626941819e-07
